In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/XiaoXiang_project')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
from flask import Flask,render_template,url_for,request
import pickle
import re
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from joblib import dump, load
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
import jieba
import warnings
warnings.filterwarnings("ignore")

In [9]:
class UserProfiling():
  def __init__(self, classifier=MultinomialNB()):
  # def __init__(self, classifier=LGBMClassifier()):
    self.classifier = classifier
    self.vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2, max_features=40000, preprocessor=self._preprocess_text)
  
  # 读取停用词表
  def get_stopwords(self, stopwords_path='./data/cn_stopwords.txt'):
    return pd.read_table(stopwords_path, header=None)[0].values

  # 私有函数，数据清洗
  def _preprocess_text(self, document):
    stopwords = self.get_stopwords()
    noise_pattern = re.compile("|".join(["http\S+", ":http\S+", "ｗｗｗ.+?\t",'\d+\@\S+']))
    clean_text = re.sub(noise_pattern, "", document)
    segs = jieba.lcut(clean_text)
    segs = list(filter(lambda x:len(x)>1, segs)) #没有解析出来的新闻过滤掉
    segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
    return ' '.join(segs)

  # 构建特征
  def features(self, X):
    return self.vectorizer.transform(X)
  
  # 拟合数据
  def fit(self, X, y, n_splits=5, shuffle=True, random_state=None):
  # def fit(self, X, y):  
    self.vectorizer.fit(X)
    # self.classifier.fit(self.features(X), y)
    self.stratified_kfold(self.features(X), y, n_splits=n_splits, shuffle=shuffle, random_state=random_state)

  # 交叉验证
  def stratified_kfold(self, X, y, n_splits, shuffle, random_state):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
    for idx, (train_index, valid_index) in enumerate(skf.split(X, y)):
      X_train, x_valid = X[train_index], X[valid_index]
      y_train, y_valid = y[train_index], y[valid_index]
      self.classifier.fit(X_train, y_train)
      print('第{}折：acc = {}'.format(idx+1, accuracy_score(y_valid, self.classifier.predict(x_valid))))

  # 预测
  def predict(self, x):
    return self.classifier.predict(self.features([x]))

  def score(self, X, y):
    return self.classifier.score(self.features(X), y)

  # 模型持久化存储
  def save_model(self, path):
    dump((self.classifier, self.vectorizer), path)
    
  # 模型加载
  def load_model(self, path):
    self.classifier, self.vectorizer = load(path)

# 模型训练与存储

In [ ]:
# 读取数据
train_df = pd.read_csv('./data/train_cutwords.csv')

train_df = train_df[~(train_df['Age']==0)]
train_df = train_df[~(train_df['Gender']==0)]
train_df = train_df[~(train_df['Education']==0)]


X = train_df['Query_List'].values[: 2000]
y_Age = train_df['Age'].values[: 2000]
y_Gender = train_df['Gender'].values[: 2000]
y_Edu = train_df['Education'].values[: 2000]


# X = train_df['Query_List'].values
# y_Age = train_df['Age'].values
# y_Gender = train_df['Gender'].values
# y_Edu = train_df['Education'].values
# num_dic = {0: 6, 1: 2, 2: 6}
# label_dict = {0: 'Age', 1: 'Gender', 2: 'Edu'}
# for idx, y in enumerate([y_Age, y_Gender, y_Edu]):
#   save_model_path = f'./Flask-APP/model/user_profiling_{label_dict[idx]}.model'
#   print(f'开始训练{label_dict[idx]}分类器。。。。')
#   user_profiling = UserProfiling()
#   user_profiling.fit(X=X, y=y)
#   # user_profiling.fit(X=X, y=y, n_split2s=5, shuffle=True, random_state=42)
#   # print('训练完成，保存模型。。。')
#   # user_profiling.save_model(save_model_path)
#   # print('模型保存完毕。。。')
#   print('-' * 20)
#   del user_profiling
age_clf = UserProfiling()
age_clf.fit(X, y_Age)

gengder_clf = UserProfiling()
gengder_clf.fit(X, y_Gender)

LightGBMError: ignored

In [ ]:
v# save_model_path = './Flask-APP/model/user_profiling_Age.model'
# print('开始训练Age分类器。。。。')
# user_profiling = UserProfiling()
# user_profiling.fit(X=X, y=y_Age, n_splits=5, shuffle=True, random_state=42)
# print('训练完成，保存模型。。。')
# user_profiling.save_model(save_model_path)
# print('模型保存完毕。。。')

In [ ]:
# save_model_path = './Flask-APP/model/user_profiling_Gender.model'
# print('开始训练Gender。。。。')
# user_profiling = UserProfiling()
# user_profiling.fit(X=X, y=y_Gender, n_splits=5, shuffle=True, random_state=42)
# print('训练完成，保存模型。。。')
# user_profiling.save_model(save_model_path)
# print('模型保存完毕。。。')
# print('-' * 20)

In [ ]:
# 读取数据
train_df = pd.read_csv('./data/train_cutwords.csv')

train_df = train_df[~(train_df['Age']==0)]
train_df = train_df[~(train_df['Gender']==0)]
train_df = train_df[~(train_df['Education']==0)]


# X = train_df['Query_List'].values[: 2000]
# y_Age = train_df['Age'].values[: 2000]
# y_Gender = train_df['Gender'].values[: 2000]
# y_Edu = train_df['Education'].values[: 2000]


X = train_df['Query_List'].values
y_Age = train_df['Age'].values
y_Gender = train_df['Gender'].values
y_Edu = train_df['Education'].values

save_model_path = './Flask-APP/model/user_profiling_Edu.model'
print('开始训练Edu。。。。')
user_profiling = UserProfiling()
user_profiling.fit(X=X, y=y_Edu, n_splits=5, shuffle=True, random_state=42)
print('训练完成，保存模型。。。')
user_profiling.save_model(save_model_path)
print('模型保存完毕。。。')
print('-' * 20)

开始训练Edu。。。。


Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.194 seconds.
Prefix dict has been built successfully.


第1折：acc = 0.5933103565173173
第2折：acc = 0.5958528730436747
第3折：acc = 0.5945304554186914
第4折：acc = 0.5911402418352356
第5折：acc = 0.596169058650695
训练完成，保存模型。。。
模型保存完毕。。。
--------------------


In [ ]:
label_dict = {0: 'Age', 1: 'Gender', 2: 'Edu'}
for k, v in label_dict.items():
  model_path = f'./Flask-APP/model/user_profiling_{v}.model'
  print(f'开始预测{v}...')
  new_user_profiling = UserProfiling()
  new_user_profiling.load_model(model_path)
  result = new_user_profiling.predict('陈学冬将出的作品	刘昊然与谭松韵	211学校的分数线	谁唱的味道好听	吻戏是真吻还是假吻	搞笑的电视剧排行榜	陈学冬身高	中南大学	南京天气	应用气象学专业	大气科学专业大学排行	黄静茵	苹果范冰冰佟大为电影完整	中国大学排名2016最新排名	南京理工大学	陈学冬整容前后	云南大学	苏州大学重点学科	搞笑的电影排行榜	韩剧中的学生时代	朴海镇	中国科学技术大学	微微一笑很倾城有多少吻戏	关晓彤只嫁李易峰	什么韩剧好看2016	南京理工大学专业	能源与动力工程专业	奶酪陷阱	青云志有多少级集	苏州大学专业排名榜	陈学冬的鼻子是真的吗	中国女排最新消息	兰州大学	软件工程专业	苹果	李易峰的现任女友	模特	杀了我治愈我	陈学冬的初吻给了谁	华东理工大学	麻雀结局	黄静茵演过的电视剧	地理科学专业	南京大学的专业	陈学冬的鼻子	同济大学	档案学专业	复旦大学	李易峰与杨幂真吻戏床	民族学专业	苏州大学类似的大学	湖南工业大学	数学与应用数学专业	地球物理学专业排名	长沙理工大学	苏州大学	青云志的主题曲	地理物理学专业	烦恼图片	大气科学专业大学	陈学冬的女朋友	陈学东是尚先生	交通工程专业	南京大学录取分数线	机械工程专业	李易峰的初吻给了谁	国际政治专业	金融学专业	奥运会开幕式	中国地质大学	刘翔110米栏世界纪录	北京爱情故事欧阳娜娜	范冰冰不雅	南京师范大学	社会学专业	中国前一百所大学排名	电气工程及其自动化专业	忧伤的歌曲排行榜	大气科学专业	上海理工大学	陈学冬	宁泽涛	微微一笑很倾城在哪个软件播	李易峰的鼻子怎么了	陈学东吻戏	经济学专业	李易峰	微微一笑很倾城电影	南京大学	中山大学	里约残奥会奖牌榜	微微一笑很倾城电视剧什么时候播	侏儒症	谭松韵	范冰冰	西南大学	苏州大学专业有哪些	汤芳	她很漂亮的男主角是谁	麻雀	南京信息工程大学	北京爱情故事	中国人民大学	刘翔	青云志碧瑶在几集死的	西南交通大学	篮球里约奥运会	土木工程专业	苏州大学专业排名	狂人日记	有那些专业	南京理工大学的分数线	物理学专业	华南理工大学	211大学名单	内衣尺寸	孙杨	金陵大学	南京医科大学	朴叙俊的作品	生物科学专业	陈翔的女朋友')
  print(f'{v}={result}')

开始预测Age...
Age=[1]
开始预测Gender...


In [ ]:
# # 读取数据
# train_df = pd.read_csv('./data/train_cutwords.csv')

# train_df = train_df[~(train_df['Age']==0)]
# train_df = train_df[~(train_df['Gender']==0)]
# train_df = train_df[~(train_df['Education']==0)]


# X = train_df['Query_List'].values[:2000]
# y_age = train_df['Age'].values[:2000]
# y_gender = train_df['Gender'].values[:2000]
# y_edu = train_df['Education'].values[:2000]
# label_dic = {0: 'Age', 1: 'Gender', 2: 'Edu'}

# save_model_path = './Flask-APP/model/user_profiling_Age.model'
# print('开始训练Age。。。。')
# x_train, x_test, y_train, y_test = train_test_split(X, y_age, random_state=42)
# user_profiling = UserProfiling()
# user_profiling.fit(X=x_train, y=y_train)
# print('验证集得分：{}'.format(user_profiling.score(x_test, y_test)))
# print('训练完成，保存模型。。。')
# user_profiling.save_model(save_model_path)
# print('模型保存完毕。。。')
# print('-' * 20)

# 模型加载

In [10]:
# 模型加载
age_path = './Flask-APP/model/user_profiling_Age.model'
edu_path = './Flask-APP/model/user_profiling_Edu.model'
gender_path = './Flask-APP/model/user_profiling_Gender.model'


user_profiling_age = UserProfiling()
user_profiling_age.load_model(age_path)

user_profiling_edu = UserProfiling()
user_profiling_edu.load_model(edu_path)

user_profiling_gender = UserProfiling()
user_profiling_gender.load_model(gender_path)